<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import numpy as np 
import pandas as pd 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv('../input/train.csv')
print(df.shape)


(42000, 785)


In [4]:
y = df['label'].values
X = df.drop(['label'],1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [5]:
print(y_test.shape)

(6300,)


In [6]:
BATCH_SIZE = 32

torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train, torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test, torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

In [7]:
torch_X_train

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [8]:
torch_X_train.dtype

torch.int64

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(784,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,10)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 

In [10]:
mlp = MLP()
print(mlp)

MLP(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)


In [54]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            #print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.data.item(), float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
                

In [55]:
fit(mlp, train_loader)

Epoch : 0 [0/35700 (0%)]	Loss: 0.249096	 Accuracy:93.750%
Epoch : 0 [1600/35700 (4%)]	Loss: 0.288773	 Accuracy:96.998%
Epoch : 0 [3200/35700 (9%)]	Loss: 0.052476	 Accuracy:97.123%
Epoch : 0 [4800/35700 (13%)]	Loss: 0.409431	 Accuracy:96.813%
Epoch : 0 [6400/35700 (18%)]	Loss: 0.177817	 Accuracy:96.346%
Epoch : 0 [8000/35700 (22%)]	Loss: 0.000140	 Accuracy:96.178%
Epoch : 0 [9600/35700 (27%)]	Loss: 0.139710	 Accuracy:96.200%
Epoch : 0 [11200/35700 (31%)]	Loss: 0.016656	 Accuracy:96.225%
Epoch : 0 [12800/35700 (36%)]	Loss: 0.075822	 Accuracy:96.189%
Epoch : 0 [14400/35700 (40%)]	Loss: 0.290737	 Accuracy:96.161%
Epoch : 0 [16000/35700 (45%)]	Loss: 0.080755	 Accuracy:96.170%
Epoch : 0 [17600/35700 (49%)]	Loss: 0.093492	 Accuracy:96.177%
Epoch : 0 [19200/35700 (54%)]	Loss: 0.029989	 Accuracy:96.287%
Epoch : 0 [20800/35700 (58%)]	Loss: 0.133300	 Accuracy:96.304%
Epoch : 0 [22400/35700 (63%)]	Loss: 0.051164	 Accuracy:96.322%
Epoch : 0 [24000/35700 (67%)]	Loss: 0.113136	 Accuracy:96.305%
Epoch

In [56]:
def evaluate(model):
#model = mlp
    correct = 0 
    for test_imgs, test_labels in test_loader:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))

In [57]:
evaluate(mlp)

Test accuracy:0.966% 
